In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.offsetbox as offsetbox
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Polygon
from pathlib import Path
import argparse

In [11]:
def plot_endcap_Ivan(ax, geometry, station, ratio=0.95):
    station_geometry = geometry[geometry['roll_name'].str.startswith(station)]
    color = 'black'
    for i in range(0, station_geometry.shape[0]):
        chamber = station_geometry.iloc[i]
        center_x = (chamber.x1 + chamber.x2 + chamber.x3 + chamber.x4) / 4
        center_y = (chamber.y1 + chamber.y2 + chamber.y3 + chamber.y4) / 4
        center_z = (chamber.z1 + chamber.z2 + chamber.z3 + chamber.z4) / 4

        p1_p2_x = np.linspace((chamber.x1 * ratio + center_x * (1 - ratio)), (chamber.x2 * ratio + center_x * (1 - ratio)), 10)
        p1_p2_y = np.linspace((chamber.y1 * ratio + center_y * (1 - ratio)), (chamber.y2 * ratio + center_y * (1 - ratio)), 10)
        p2_p3_x = np.linspace((chamber.x2 * ratio + center_x * (1 - ratio)), (chamber.x3 * ratio + center_x * (1 - ratio)), 10)
        p2_p3_y = np.linspace((chamber.y2 * ratio + center_y * (1 - ratio)), (chamber.y3 * ratio + center_y * (1 - ratio)), 10)
        p3_p4_x = np.linspace((chamber.x3 * ratio + center_x * (1 - ratio)), (chamber.x4 * ratio + center_x * (1 - ratio)), 10)
        p3_p4_y = np.linspace((chamber.y3 * ratio + center_y * (1 - ratio)), (chamber.y4 * ratio + center_y * (1 - ratio)), 10)
        p4_p1_x = np.linspace((chamber.x4 * ratio + center_x * (1 - ratio)), (chamber.x1 * ratio + center_x * (1 - ratio)), 10)
        p4_p1_y = np.linspace((chamber.y4 * ratio + center_y * (1 - ratio)), (chamber.y1 * ratio + center_y * (1 - ratio)), 10)

        p_x = np.concatenate((p1_p2_x, p2_p3_x, p3_p4_x, p4_p1_x))
        p_y = np.concatenate((p1_p2_y, p2_p3_y, p3_p4_y, p4_p1_y))

        endcap_chamber_name = chamber.roll_name  
        ax.text(center_x, center_y,
                endcap_chamber_name[5:] + f'\n{center_z:.1f}',
                fontsize=10, c=color, va='center', ha='center', weight='bold')

        ax.plot(p_x, p_y, c=color, lw=3)
    ax.plot([], [], c=color, label='Installed chamber from Ivan', lw=3)
    return ax

def plot_endcap_cmssw(ax, geometry, station, ratio=0.95):
    station_geometry = geometry[geometry['roll_name'].str.startswith(station)]
    for i in range(0, station_geometry.shape[0]):
        chamber = station_geometry.iloc[i]
        center_x = (chamber.x1 + chamber.x2 + chamber.x3 + chamber.x4) / 4
        center_y = (chamber.y1 + chamber.y2 + chamber.y3 + chamber.y4) / 4
        center_z = (chamber.z1 + chamber.z2 + chamber.z3 + chamber.z4) / 4

        p1_p2_x = np.linspace((chamber.x1 * ratio + center_x * (1 - ratio)), (chamber.x2 * ratio + center_x * (1 - ratio)), 10)
        p1_p2_y = np.linspace((chamber.y1 * ratio + center_y * (1 - ratio)), (chamber.y2 * ratio + center_y * (1 - ratio)), 10)
        p2_p3_x = np.linspace((chamber.x2 * ratio + center_x * (1 - ratio)), (chamber.x3 * ratio + center_x * (1 - ratio)), 10)
        p2_p3_y = np.linspace((chamber.y2 * ratio + center_y * (1 - ratio)), (chamber.y3 * ratio + center_y * (1 - ratio)), 10)
        p3_p4_x = np.linspace((chamber.x3 * ratio + center_x * (1 - ratio)), (chamber.x4 * ratio + center_x * (1 - ratio)), 10)
        p3_p4_y = np.linspace((chamber.y3 * ratio + center_y * (1 - ratio)), (chamber.y4 * ratio + center_y * (1 - ratio)), 10)
        p4_p1_x = np.linspace((chamber.x4 * ratio + center_x * (1 - ratio)), (chamber.x1 * ratio + center_x * (1 - ratio)), 10)
        p4_p1_y = np.linspace((chamber.y4 * ratio + center_y * (1 - ratio)), (chamber.y1 * ratio + center_y * (1 - ratio)), 10)

        p_x = np.concatenate((p1_p2_x, p2_p3_x, p3_p4_x, p4_p1_x))
        p_y = np.concatenate((p1_p2_y, p2_p3_y, p3_p4_y, p4_p1_y))

        
        endcap_chamber_name = chamber.roll_name.split('_') # [RE+3, R3, CH29, A]
        if endcap_chamber_name[1][1] == "1":
            ax.text(center_x, center_y,
                    f'{endcap_chamber_name[1][-1]}/{endcap_chamber_name[2][2:]}/{endcap_chamber_name[3]}' + f'\n{center_z:.1f}', # 3/29
                    fontsize=6,
                    va='center', ha='center', alpha=0.8)
        else:        
            ax.text(center_x, center_y,
                    f'{endcap_chamber_name[1][-1]}/{endcap_chamber_name[2][2:]}/{endcap_chamber_name[3]}' + f'\n{center_z:.1f}', # 3/29
                    fontsize=7,
                    va='center', ha='center', alpha=0.8)

        if chamber.is_front == True:
            ax.plot(p_x, p_y, c="red", alpha=0.6)
        if chamber.is_front == False:
            ax.plot(p_x, p_y, c="blue", alpha=0.6)
    ax.plot([], [], c='red', label='Front roll from CMSSW')
    ax.plot([], [], c='blue', label='Back roll from CMSSW')
    return ax
    
def plot_endcap_cmssw_lr(ax, geometry, station, ratio=0.95):
    station_geometry = geometry[geometry['roll_name'].str.startswith(station)]
    for i in range(0, station_geometry.shape[0]):
        chamber = station_geometry.iloc[i]
        center_x = (chamber.x1 + chamber.x2 + chamber.x3 + chamber.x4) / 4
        center_y = (chamber.y1 + chamber.y2 + chamber.y3 + chamber.y4) / 4
        center_z = (chamber.z1 + chamber.z2 + chamber.z3 + chamber.z4) / 4

        p1_p2_x = np.linspace((chamber.x1 * ratio + center_x * (1 - ratio)), (chamber.x2 * ratio + center_x * (1 - ratio)), 10)
        p1_p2_y = np.linspace((chamber.y1 * ratio + center_y * (1 - ratio)), (chamber.y2 * ratio + center_y * (1 - ratio)), 10)
        p2_p3_x = np.linspace((chamber.x2 * ratio + center_x * (1 - ratio)), (chamber.x3 * ratio + center_x * (1 - ratio)), 10)
        p2_p3_y = np.linspace((chamber.y2 * ratio + center_y * (1 - ratio)), (chamber.y3 * ratio + center_y * (1 - ratio)), 10)
        p3_p4_x = np.linspace((chamber.x3 * ratio + center_x * (1 - ratio)), (chamber.x4 * ratio + center_x * (1 - ratio)), 10)
        p3_p4_y = np.linspace((chamber.y3 * ratio + center_y * (1 - ratio)), (chamber.y4 * ratio + center_y * (1 - ratio)), 10)
        p4_p1_x = np.linspace((chamber.x4 * ratio + center_x * (1 - ratio)), (chamber.x1 * ratio + center_x * (1 - ratio)), 10)
        p4_p1_y = np.linspace((chamber.y4 * ratio + center_y * (1 - ratio)), (chamber.y1 * ratio + center_y * (1 - ratio)), 10)

        ax.plot(p1_p2_x, p1_p2_y, c='black')
        ax.plot(p2_p3_x, p2_p3_y, c='red')
        ax.plot(p3_p4_x, p3_p4_y, c='black')
        ax.plot(p4_p1_x, p4_p1_y, c='blue')
        
        endcap_chamber_name = chamber.roll_name.split('_') # [RE+3, R3, CH29, A]
        if endcap_chamber_name[1][1] == "1":
            ax.text(center_x, center_y,
                    f'{endcap_chamber_name[1][-1]}/{endcap_chamber_name[2][2:]}/{endcap_chamber_name[3]}' + f'\n{center_z:.1f}', # 3/29
                    fontsize=6,
                    va='center', ha='center', alpha=0.8)
        else:        
            ax.text(center_x, center_y,
                    f'{endcap_chamber_name[1][-1]}/{endcap_chamber_name[2][2:]}/{endcap_chamber_name[3]}' + f'\n{center_z:.1f}', # 3/29
                    fontsize=7,
                    va='center', ha='center', alpha=0.8)


    ax.plot([], [], c='red', label='p2_p3')
    ax.plot([], [], c='blue', label='p4_p1')
    return ax

In [ ]:
workspace = Path('/afs/cern.ch/user/j/joshin/workspace-lxplus/rpc-geom/workspace-rpc-geom/250304-plotting')
source_list = [
    'rpcf-2015-v1', 'rpcf-2015-v2',
    'rpcf-2021-v2', 'rpcf-2021-v4',
    'rpcf-2023-v2',
    'rpcf-2024-v1', 'rpcf-2024-v3',
    'rpcf-2025-v1',
    'rpcf-2030-v3', 'rpcf-2030-v4'
]

geometry_Ivan = pd.read_csv((workspace / 'csv' / 'Ivan-240722').with_suffix('.csv'), sep=',', index_col=False)
for source in source_list:    
    fb_output_dir = workspace / 'plot' / 'full-fb' / source
    lr_output_dir = workspace / 'plot' / 'full-lr' / source
    geom_dir = workspace / 'csv' / source
    geometry = pd.read_csv(geom_dir.with_suffix('.csv'), sep=',', index_col=False)
    for station in ['RE-1', 'RE-2', 'RE-3', 'RE-4', 'RE+1', 'RE+2', 'RE+3', 'RE+4']:
        # Plotting RPC geometry: Front Back
        fig, ax = plt.subplots(figsize=(12.8, 10.24))        
        plot_endcap_cmssw(ax, geometry, station, ratio=1.0)
        plot_endcap_Ivan(ax, geometry_Ivan, station, ratio=1.0)
        ax.set_title(f'RPC {station} Geometry', fontsize=18, weight='bold')
        ax.set_xlabel('Global X [cm]', fontsize=16)
        ax.set_ylabel('Global Y [cm]', fontsize=16)
        ax.legend(fontsize=14, loc='upper right')
        fig.tight_layout()
        if not fb_output_dir.exists(): 
            fb_output_dir.mkdir(parents=True)
        fig.savefig(f'{fb_output_dir}/{station}.png')
        plt.close(fig)

        # Plotting RPC geometry: Left Right
        fig, ax = plt.subplots(figsize=(12.8, 10.24))   
        plot_endcap_cmssw_lr(ax, geometry, station, ratio=0.9)
        ax.set_title(f'RPC {station} Geometry', fontsize=18, weight='bold')
        ax.legend(fontsize=14, loc='upper right')
        ax.set_xlabel('Global X [cm]', fontsize=16)
        ax.set_ylabel('Global Y [cm]', fontsize=16)
        plt.tight_layout()
        if not lr_output_dir.exists():
            lr_output_dir.mkdir(parents=True)
        fig.savefig(f'{lr_output_dir}/{station}.png')
        plt.close(fig)